In [7]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
import scipy.sparse
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

In [10]:
df = pd.read_csv("../3. Data Cleaning/processed_merged_carparks.csv")

In [11]:
df

,carparkNo,lotsAvailable,lotType,weather,description,temperature,humidity,ingestionDateTime,weekdayRate,satdayRate,...,distance_to_nearest_primary_school__km_,distance_to_nearest_secondary_school__km_,FilledProportion,hour,weekday,weekend,day,strippedEndTime,strippedStartTime,rate
0,S0055,0.0,C,Clouds,broken clouds,30.07,67.0,2023-03-12 13:00:00,0.75,0.75,...,0.641623,0.547748,1.000000,13,6,1,12,14,7,0.75
1,S0055,1.0,C,Clouds,broken clouds,30.86,72.0,2023-03-12 14:00:00,0.75,0.75,...,0.641623,0.547748,0.990476,14,6,1,12,14,7,0.75
2,S0055,4.0,C,Clouds,broken clouds,29.44,73.0,2023-03-09 14:00:00,0.75,0.75,...,0.641623,0.547748,0.961905,14,3,0,9,14,7,0.75
3,S0055,0.0,C,Clouds,broken clouds,29.56,73.0,2023-03-10 14:00:00,0.75,0.75,...,0.641623,0.547748,1.000000,14,4,0,10,14,7,0.75
4,S0055,0.0,C,Clouds,broken clouds,29.20,74.0,2023-03-08 13:00:00,0.75,0.75,...,0.641623,0.547748,1.000000,13,2,0,8,14,7,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22434,B0032,83.0,C,Clouds,scattered clouds,25.11,92.0,2023-03-09 07:00:00,0.60,0.70,...,0.073367,1.053293,0.290598,7,3,0,9,11,7,0.60
22435,B0032,7.0,C,Clouds,scattered clouds,27.73,79.0,2023-03-16 10:00:00,0.60,0.70,...,0.073367,1.053293,0.940171,10,3,0,16,11,7,0.60
22436,B0032,10.0,C,Clouds,scattered clouds,25.74,83.0,2023-03-15 09:00:00,0.60,0.70,...,0.073367,1.053293,0.914530,9,2,0,15,11,7,0.60
22437,B0032,68.0,C,Clouds,scattered clouds,24.91,85.0,2023-03-16 07:00:00,0.60,0.70,...,0.073367,1.053293,0.418803,7,3,0,16,11,7,0.60


In [12]:
#one hot encode categorical variables
enc = OneHotEncoder(handle_unknown='ignore')
X_enc = enc.fit_transform(df[['weather']])
enc.categories_

[array(['Clouds', 'Rain'], dtype=object)]

In [13]:
X_enc = X_enc.todense()
print(type(X_enc))

<class 'numpy.matrix'>


In [14]:
X_enc_d = enc.fit_transform(df[['description']])
enc.categories_

[array(['broken clouds', 'few clouds', 'light rain', 'overcast clouds',
        'scattered clouds'], dtype=object)]

In [15]:
X_enc_d = X_enc_d.todense()

In [16]:
df_temp = pd.DataFrame(X_enc, columns = ['Clouds', 'Rain'])
df_temp_d = pd.DataFrame(X_enc_d, columns = ['broken clouds', 'few clouds', 'light rain', 'overcast clouds','scattered clouds'])

In [17]:
df = pd.concat([df,df_temp],axis=1)
df = pd.concat([df,df_temp_d],axis=1)

In [18]:
X_enc_m = enc.fit_transform(df[['lotType']])
enc.categories_

[array(['C', 'H', 'M'], dtype=object)]

In [19]:
X_enc_m = X_enc_m.todense()

In [20]:
df_temp = pd.DataFrame(X_enc_m, columns = ['C', 'H', 'M'])

In [21]:
df = pd.concat([df,df_temp],axis=1)

In [36]:
df = df[df['lotType'] == "C"]

In [23]:
df.columns

Index(['carparkNo', 'lotsAvailable', 'lotType', 'weather', 'description',
       'temperature', 'humidity', 'ingestionDateTime', 'weekdayRate',
       'satdayRate', 'sunPHRate', 'startTime', 'totalCapacity', 'endTime',
       'latitude', 'longitude', 'distance_to_cbd__km_',
       'distance_to_nearest_shopping_mall__km_',
       'distance_to_nearest_university__km_',
       'distance_to_nearest_pre_school__km_',
       'distance_to_nearest_primary_school__km_',
       'distance_to_nearest_secondary_school__km_', 'FilledProportion', 'hour',
       'weekday', 'weekend', 'day', 'strippedEndTime', 'strippedStartTime',
       'rate', 'Clouds', 'Rain', 'broken clouds', 'few clouds', 'light rain',
       'overcast clouds', 'scattered clouds', 'C', 'H', 'M'],
      dtype='object')

In [37]:
df

,carparkNo,lotsAvailable,lotType,weather,description,temperature,humidity,ingestionDateTime,weekdayRate,satdayRate,...,Clouds,Rain,broken clouds,few clouds,light rain,overcast clouds,scattered clouds,C,H,M
0,S0055,0.0,C,Clouds,broken clouds,30.07,67.0,2023-03-12 13:00:00,0.75,0.75,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,S0055,1.0,C,Clouds,broken clouds,30.86,72.0,2023-03-12 14:00:00,0.75,0.75,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,S0055,4.0,C,Clouds,broken clouds,29.44,73.0,2023-03-09 14:00:00,0.75,0.75,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,S0055,0.0,C,Clouds,broken clouds,29.56,73.0,2023-03-10 14:00:00,0.75,0.75,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,S0055,0.0,C,Clouds,broken clouds,29.20,74.0,2023-03-08 13:00:00,0.75,0.75,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22434,B0032,83.0,C,Clouds,scattered clouds,25.11,92.0,2023-03-09 07:00:00,0.60,0.70,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
22435,B0032,7.0,C,Clouds,scattered clouds,27.73,79.0,2023-03-16 10:00:00,0.60,0.70,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
22436,B0032,10.0,C,Clouds,scattered clouds,25.74,83.0,2023-03-15 09:00:00,0.60,0.70,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
22437,B0032,68.0,C,Clouds,scattered clouds,24.91,85.0,2023-03-16 07:00:00,0.60,0.70,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [38]:
# Drop columns that have been used to derive features
df_temp = df.drop(columns=['carparkNo','weather','description','strippedEndTime','strippedStartTime','lotType',
                           'ingestionDateTime','startTime','endTime',
                          'satdayRate','sunPHRate','weekdayRate'])
df_temp

,lotsAvailable,temperature,humidity,totalCapacity,latitude,longitude,distance_to_cbd__km_,distance_to_nearest_shopping_mall__km_,distance_to_nearest_university__km_,distance_to_nearest_pre_school__km_,...,Clouds,Rain,broken clouds,few clouds,light rain,overcast clouds,scattered clouds,C,H,M
0,0.0,30.07,67.0,105,1.362800,103.866326,8.876435,10.870413,6.813202,0.698084,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,30.86,72.0,105,1.362800,103.866326,8.876435,10.870413,6.813202,0.698084,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,4.0,29.44,73.0,105,1.362800,103.866326,8.876435,10.870413,6.813202,0.698084,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,29.56,73.0,105,1.362800,103.866326,8.876435,10.870413,6.813202,0.698084,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,29.20,74.0,105,1.362800,103.866326,8.876435,10.870413,6.813202,0.698084,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22434,83.0,25.11,92.0,117,1.339089,103.776396,10.343492,6.435652,1.191936,0.409950,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
22435,7.0,27.73,79.0,117,1.339089,103.776396,10.343492,6.435652,1.191936,0.409950,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
22436,10.0,25.74,83.0,117,1.339089,103.776396,10.343492,6.435652,1.191936,0.409950,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
22437,68.0,24.91,85.0,117,1.339089,103.776396,10.343492,6.435652,1.191936,0.409950,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [39]:
#dropping highly correlated variables
df_temp = df_temp.drop(columns=['temperature'])
df_temp


,lotsAvailable,humidity,totalCapacity,latitude,longitude,distance_to_cbd__km_,distance_to_nearest_shopping_mall__km_,distance_to_nearest_university__km_,distance_to_nearest_pre_school__km_,distance_to_nearest_primary_school__km_,...,Clouds,Rain,broken clouds,few clouds,light rain,overcast clouds,scattered clouds,C,H,M
0,0.0,67.0,105,1.362800,103.866326,8.876435,10.870413,6.813202,0.698084,0.641623,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,72.0,105,1.362800,103.866326,8.876435,10.870413,6.813202,0.698084,0.641623,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,4.0,73.0,105,1.362800,103.866326,8.876435,10.870413,6.813202,0.698084,0.641623,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,73.0,105,1.362800,103.866326,8.876435,10.870413,6.813202,0.698084,0.641623,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,74.0,105,1.362800,103.866326,8.876435,10.870413,6.813202,0.698084,0.641623,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22434,83.0,92.0,117,1.339089,103.776396,10.343492,6.435652,1.191936,0.409950,0.073367,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
22435,7.0,79.0,117,1.339089,103.776396,10.343492,6.435652,1.191936,0.409950,0.073367,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
22436,10.0,83.0,117,1.339089,103.776396,10.343492,6.435652,1.191936,0.409950,0.073367,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
22437,68.0,85.0,117,1.339089,103.776396,10.343492,6.435652,1.191936,0.409950,0.073367,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [40]:
# split into train test set
testsize = 0.2  
seed = 0
X = df_temp.drop('FilledProportion', axis = 1)
Y = df_temp['FilledProportion']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)


(13081, 26) (3271, 26)
(13081,) (3271,)


In [41]:
#simple Regression Model

from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train, Y_train)

LinearRegression()

In [42]:
Y_fit = lm.predict(X_test)
Y_fit_2 = lm.predict(X_train)
print(Y_fit.shape)
print(Y_test.shape)

(3271,)
(3271,)


In [43]:
from sklearn.metrics import r2_score, mean_squared_error

print('R-squared:', r2_score(Y_test, Y_fit))
print('MSE testing:', mean_squared_error(Y_test, Y_fit))
print('MSE training:', mean_squared_error(Y_train, Y_fit_2))
print('MAE:', mean_absolute_error(Y_test, Y_fit))

R-squared: 0.592218341839048
MSE testing: 0.06120494772877179
MSE training: 0.062444099072731615
MAE: 0.18386868436831647


In [44]:
from sklearn.tree import DecisionTreeRegressor
lm2 = DecisionTreeRegressor()
lm2.fit(X_train, Y_train)
Y_fit = lm2.predict(X_test)
Y_fit_2 = lm2.predict(X_train)

print('R-squared:', r2_score(Y_test, Y_fit))
print('MSE testing:', mean_squared_error(Y_test, Y_fit))
print('MSE training:', mean_squared_error(Y_train, Y_fit_2))
print('MAE:', mean_absolute_error(Y_test, Y_fit))

R-squared: 0.9994326114941524
MSE testing: 8.516073037449433e-05
MSE training: 4.025487006497747e-31
MAE: 0.0019808910720353853


In [45]:
from sklearn.ensemble import GradientBoostingRegressor
lm3 = GradientBoostingRegressor()
lm3.fit(X_train, Y_train)
Y_fit = lm3.predict(X_test)
Y_fit_2 = lm3.predict(X_train)

print('R-squared:', r2_score(Y_test, Y_fit))
print('MSE testing:', mean_squared_error(Y_test, Y_fit))
print('MSE training:', mean_squared_error(Y_train, Y_fit_2))
print('MAE:', mean_absolute_error(Y_test, Y_fit))

R-squared: 0.9909314303583286
MSE testing: 0.0013611238264035553
MSE training: 0.0014282970111506449
MAE: 0.023746186287209134
